In [79]:
from random import randint
from sklearn.utils import shuffle

train_X = []
train_label = []

def generate_random_data(n_sample, min_age, max_age, label, X, label_data):
    for i in range(n_sample):
        X.append(randint(min_age, max_age))
        label_data.append(label)

generate_random_data(50, 1, 64, 1, train_X, train_label)
generate_random_data(50, 65, 100, 0, train_X, train_label)
generate_random_data(1000, 1, 64, 0, train_X, train_label)
generate_random_data(1000, 65, 100, 1, train_X, train_label)

train_X, train_label = shuffle(train_X, train_label)

from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler(feature_range=(0,1))
train_X = np.array(train_X)
train_y = np.array(train_label)
train_X = scaler.fit_transform(train_X.reshape(-1,1))

In [80]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [81]:
model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [82]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 16)                32        
                                                                 
 dense_28 (Dense)            (None, 32)                544       
                                                                 
 dense_29 (Dense)            (None, 2)                 66        
                                                                 
Total params: 642 (2.51 KB)
Trainable params: 642 (2.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [83]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [84]:
model.fit(x=train_X, y=train_y, validation_split=0.1, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30
189/189 - 1s - loss: 0.6922 - accuracy: 0.5481 - val_loss: 0.6861 - val_accuracy: 0.4952 - 1s/epoch - 6ms/step
Epoch 2/30
189/189 - 0s - loss: 0.6705 - accuracy: 0.5894 - val_loss: 0.6704 - val_accuracy: 0.5619 - 197ms/epoch - 1ms/step
Epoch 3/30
189/189 - 0s - loss: 0.6444 - accuracy: 0.6593 - val_loss: 0.6500 - val_accuracy: 0.6381 - 186ms/epoch - 984us/step
Epoch 4/30
189/189 - 0s - loss: 0.6176 - accuracy: 0.7323 - val_loss: 0.6250 - val_accuracy: 0.7286 - 191ms/epoch - 1ms/step
Epoch 5/30
189/189 - 0s - loss: 0.5905 - accuracy: 0.7688 - val_loss: 0.6034 - val_accuracy: 0.7429 - 185ms/epoch - 980us/step
Epoch 6/30
189/189 - 0s - loss: 0.5640 - accuracy: 0.7889 - val_loss: 0.5809 - val_accuracy: 0.7476 - 186ms/epoch - 985us/step
Epoch 7/30
189/189 - 0s - loss: 0.5361 - accuracy: 0.8032 - val_loss: 0.5546 - val_accuracy: 0.7667 - 189ms/epoch - 999us/step
Epoch 8/30
189/189 - 0s - loss: 0.5078 - accuracy: 0.8360 - val_loss: 0.5306 - val_accuracy: 0.7714 - 204ms/epoch - 1ms/

In [85]:
test_sample = []
test_label = []

generate_random_data(50, 1, 64, 1, test_sample, test_label)
generate_random_data(50, 65, 100, 0, test_sample, test_label)
generate_random_data(1000, 1, 64, 0, test_sample, test_label)
generate_random_data(1000, 65, 100, 1, test_sample, test_label)

test_sample, test_label = shuffle(test_sample, test_label)
test_sample = np.array(test_sample)
test_sample = scaler.fit_transform(test_sample.reshape(-1,1))

In [86]:
predictions = model.predict(x=test_sample, batch_size=10, verbose=0)

In [87]:
rounded_predictions = np.argmax(predictions, axis=-1)

In [111]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(test_label, rounded_predictions)

0.9295238095238095

In [112]:
confusion_matrix(test_label, rounded_predictions)

array([[ 948,  102],
       [  46, 1004]])

In [116]:
model.save('models/medical_trial_model.h5')

The above .save() method saves:
* The architecture of the model, allowing re-create the model with same architecture.
* The weights of the model.
* The training configuration (loss, optimizer).
* The state of the optimizer, allowing to resume training exactly where you left off.

In [117]:
from tensorflow.keras.models import load_model

loaded_model = load_model("models/medical_trial_model.h5")

In [118]:
loaded_model.weights

[<tf.Variable 'dense_30/kernel:0' shape=(1, 16) dtype=float32, numpy=
 array([[ 0.3709432 ,  0.42154488,  0.3886046 , -0.5234844 ,  0.02905918,
         -0.10985295,  0.33868766, -0.4860571 , -0.48520374,  0.44651908,
          0.18116567,  0.3695966 ,  0.26110223,  0.0327641 , -0.35487503,
          0.4570869 ]], dtype=float32)>,
 <tf.Variable 'dense_30/bias:0' shape=(16,) dtype=float32, numpy=
 array([-0.1263436 , -0.12353495, -0.08085185,  0.        ,  0.18634692,
         0.1669892 ,  0.17672668,  0.        ,  0.        , -0.11594213,
        -0.07434864, -0.11952471, -0.08741551,  0.1457539 ,  0.        ,
        -0.14016841], dtype=float32)>,
 <tf.Variable 'dense_31/kernel:0' shape=(16, 32) dtype=float32, numpy=
 array([[ 0.09829018, -0.09334318, -0.32674733, -0.4859526 ,  0.3255262 ,
          0.02911621,  0.41365817,  0.23851615,  0.17792852, -0.37242156,
         -0.32920295, -0.03259487,  0.08516192, -0.05380398,  0.10060468,
         -0.02256223,  0.25514528, -0.01873192,  0

In [119]:
loaded_model.optimizer